# STEP 0
## Implementazione del MIP Cut Searcher

## Definizione dei parametri in input
- Circuito quantistico -> rappresentato come DAG
- Massimo numero di qbit per sottocircuito = dimensione dei QPU disponibili
- Massimo numero di sottocircuito

In [21]:
# definizione degli archi del grafo
edges = [
    (0, 1), (1, 2),
    (2, 3), (3, 4),
    (4, 5), (5, 6),
    (6, 7), (7, 8),
    (8, 9),
]

# insieme dei vertici dal grafo
vertices = set()
for edge in edges:
    vertices.update(edge)

max_qubits_per_subcircuit = 4  # numero massimo di qubit per sottocircuito
num_subcircuits = 3  # numero massimo di sottocircuiti

# definizione dei pesi dei vertici (numero di qubit necessari per ogni vertice)
vertex_weights = {v: 1 for v in vertices} # 1 per test

## Definizione delle Variabili del problema

In [22]:
import pulp

# definizione del problema
problem = pulp.LpProblem("CircuitCutter", pulp.LpMinimize)

subcircuits = range(num_subcircuits)  # Indici dei sottocircuiti

# y[v,c]: indica se il gate v appartiene al sottocircuito c
y = pulp.LpVariable.dicts("y", [(v, c) for v in vertices for c in subcircuits], cat="Binary")

# x[e,c]: indica se l'arco e è tagliato dal sottocircuito c
x = pulp.LpVariable.dicts("x", [(e, c) for e in edges for c in subcircuits], cat="Binary")

# a[c]: somma di qubit originali in input per il sottocircuito c
a = pulp.LpVariable.dicts("a", subcircuits, cat="Integer")

# p[c]: somma di qubit di inizializzazione per il sottocircuito c
p = pulp.LpVariable.dicts("p", subcircuits, cat="Integer")

# o[c]: numero di qubit misurati per il sottocircuito c
o = pulp.LpVariable.dicts("o", subcircuits, cat="Integer")

# f[c]: numero di qubit del sottocircuito c che contribuiscono alla misura finale dell'intero circuito
f = pulp.LpVariable.dicts("f", subcircuits, cat="Integer")

# d[c]: totale dei qubit del sottocircuito c, calcolato come a[c] + p[c]
d = pulp.LpVariable.dicts("d", subcircuits, cat="Integer")

# variabili ausiliarie per linearizzare i prodotti
# z_p[e,  c]: variabile binaria per il prodotto relativo a p[c]
z_p = pulp.LpVariable.dicts("z_p", [(e, c) for e in edges for c in subcircuits], cat="Binary")

# z_o[e, c]: variabile binaria per il prodotto relativo a o[c]
z_o = pulp.LpVariable.dicts("z_o", [(e, c) for e in edges for c in subcircuits], cat="Binary")

# y[v, c], x[e, c] sono binarie che rappresentano la struttura del circuito e i tagli
# a[c], p[c], o[c], f[c] e d[c] sono variabili intere che descrivono i parametri del sottocircuito
# z_p e z_o sono variabili ausiliarie per linearizzare i prodotti

## Definizione dei vincoli principali del problema

In [23]:
# vincoli sui sottocircuiti
for c in subcircuits:
    problem += a[c] == pulp.lpSum(vertex_weights[v] * y[v, c] for v in vertices), f"Qbits_Originali_Sottocircuito_{c}"
    problem += o[c] == pulp.lpSum(z_o[(e, c)] for e in edges), f"Qbits_Misurazione_Sottocircuito_{c}"
    problem += f[c] == a[c] + p[c] - o[c], f"Qbits_Contribuenti_Sottocircuito_{c}"
    problem += d[c] == a[c] + p[c], f"Numero_totale_qbit_Input_Sottocircuito_{c}"
    problem += p[c] == pulp.lpSum(z_p[(e, c)] for e in edges), f"Qbits_Inizializzazione_Sottocircuito_{c}"
    problem += d[c] <= max_qubits_per_subcircuit, f"Qbit_inferiore_a_nc_{c}"

# vincoli di linearizzazione per i prodotti
for e in edges:
    for c in subcircuits:
        # linearizzazione per x[e, c] * y[e[1], c] (qubit di inizializzazione)
        problem += z_p[(e, c)] <= x[(e, c)], f"p_Linearizzazione1_{e}_{c}"
        problem += z_p[(e, c)] <= y[(e[1], c)], f"p_Linearizzazione2_{e}_{c}"
        problem += z_p[(e, c)] >= x[(e, c)] + y[(e[1], c)] - 1, f"p_Linearizzazione3_{e}_{c}"

        # linearizzazione di x[e, c] * y[e[0], c] (qubit di misurazione)
        problem += z_o[(e, c)] <= x[(e, c)], f"O_Linearizzazione1_{e}_{c}"
        problem += z_o[(e, c)] <= y[(e[0], c)], f"O_Linearizzazione2_{e}_{c}"
        problem += z_o[(e, c)] >= x[(e, c)] + y[(e[0], c)] - 1, f"O_Linearizzazione3_{e}_{c}"

# ogni vertice deve appartenere a un unico sottocircuito
for v in vertices:
    problem += pulp.lpSum(y[v, c] for c in subcircuits) == 1, f"Unico_Sottocircuito_Per_Vertice_{v}"

# vincoli sugli archi nei sottocircuiti
for c in subcircuits:
    for e in edges:
        problem += x[e, c] <= y[e[0], c] + y[e[1], c], f"Vincolo_11.1_{c}_{e}"
        problem += x[e, c] >= y[e[0], c] - y[e[1], c], f"Vincolo_11.2_{c}_{e}"
        problem += x[e, c] >= y[e[1], c] - y[e[0], c], f"Vincolo_11.3_{c}_{e}"
        problem += x[e, c] <= 2 - y[e[0], c] - y[e[1], c], f"Vincolo_11.4_{c}_{e}"

# vincolo di ordine sui sottocircuiti
for k in range(num_subcircuits):
    problem += pulp.lpSum(y[k, j] for j in range(k + 1, num_subcircuits)) == 0, f"Vincolo_Sottocircuito_{k}"


## Funzione obiettivo

in realtà la funzione obiettivo non è questa ma quella che serve a minimizzare 
il tempo di post-processing ([14] nel paper). Non so come rappresentare la funzione perchè non lineare.
Una mia idea era approssimare il valore di K, magari calcolare il  K solo in determinati
punti per approssimare 4^K mentre per quanto riguarda la produttoria non ne ho idea


In [ ]:
# definizione della funzione obiettivo
K = pulp.lpSum(x[e, c] for c in subcircuits for e in edges) / 2
problem += K, f'Minimizza_Tagli'
problem.solve()

1

## Stampa del risultato

In [25]:

print(f"Status: {pulp.LpStatus[problem.status]}\n")
print(f"Obiettivo (numero di tagli / 2): {pulp.value(problem.objective)}\n\n")

print("Assegnazione dei vertici ai sottocircuiti:\n")
for v in vertices:
    for c in subcircuits:
        if pulp.value(y[(v, c)]) == 1:
            print(f"  Vertice {v} assegnato al sottocircuito {c}\n")

print("\nTagli sugli archi:\n")
for e in edges:
    for c in subcircuits:
        if pulp.value(x[(e, c)]) == 1:
            print(f"  Arco {e} tagliato dal sottocircuito {c}\n")

print("\nValori delle variabili di qubit per sottocircuito:\n")
for c in subcircuits:
    a_val = pulp.value(a[c])
    p_val = pulp.value(p[c])
    o_val = pulp.value(o[c])
    f_val = pulp.value(f[c])
    d_val = pulp.value(d[c])
    print(f"  Sottocircuito {c}:\n")
    print(f"    a[{c}] = {a_val}  (Qubit originali inclusi)\n")
    print(f"    p[{c}] = {p_val}  (Qubit di inizializzazione)\n")
    print(f"    o[{c}] = {o_val}  (Qubit misurati in uscita)\n")
    print(f"    f[{c}] = {f_val}  (Qubit che contribuiscono alla misura finale)\n")
    print(f"    d[{c}] = {d_val}  (Totale qubit in input = a[{c}] + p[{c}])\n")



Status: Optimal

Obiettivo (numero di tagli / 2): 2.0


Assegnazione dei vertici ai sottocircuiti:

  Vertice 0 assegnato al sottocircuito 0

  Vertice 1 assegnato al sottocircuito 0

  Vertice 2 assegnato al sottocircuito 0

  Vertice 3 assegnato al sottocircuito 0

  Vertice 4 assegnato al sottocircuito 2

  Vertice 5 assegnato al sottocircuito 2

  Vertice 6 assegnato al sottocircuito 2

  Vertice 7 assegnato al sottocircuito 1

  Vertice 8 assegnato al sottocircuito 1

  Vertice 9 assegnato al sottocircuito 1


Tagli sugli archi:

  Arco (3, 4) tagliato dal sottocircuito 0

  Arco (3, 4) tagliato dal sottocircuito 2

  Arco (6, 7) tagliato dal sottocircuito 1

  Arco (6, 7) tagliato dal sottocircuito 2


Valori delle variabili di qubit per sottocircuito:

  Sottocircuito 0:

    a[0] = 4.0  (Qubit originali inclusi)

    p[0] = 0.0  (Qubit di inizializzazione)

    o[0] = 1.0  (Qubit misurati in uscita)

    f[0] = 3.0  (Qubit che contribuiscono alla misura finale)

    d[0] = 4.0 